# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('combined_merged_features.npz') 

# Access the concatenated features from the .npz file
X_merged_features = data['X_merged_features']

# Close the loaded file
data.close()

#print the merged array
X_merged_features

array([[[-1.90864583e-06, -1.01319508e-06, -1.25947210e-06, ...,
          3.05983596e-11,  5.69918532e-01,  1.43285998e+00],
        [-4.16825421e-05, -4.20260902e-05, -4.24717352e-05, ...,
          1.80793891e-11,  3.16028756e-01,  2.11285116e+00],
        [-1.90472102e-05, -1.92407524e-05, -1.70617167e-05, ...,
          2.03574174e-11,  3.68552716e-01,  1.69556851e+00],
        ...,
        [ 6.94002021e-06,  6.58408463e-06,  6.57031824e-06, ...,
          2.44995562e-11,  3.34763239e-01,  2.11152035e+00],
        [ 2.41499888e-05,  2.47479670e-05,  2.44680713e-05, ...,
          2.61209914e-11,  3.37369781e-01,  1.97140856e+00],
        [-1.60015896e-06, -3.41169756e-06, -2.49139569e-06, ...,
          7.24925208e-11,  4.56258399e-01,  1.62313244e+00]],

       [[ 4.44703238e-05,  4.35599754e-05,  4.57796671e-05, ...,
          1.79259348e-10,  2.21715280e-01,  3.09942440e+00],
        [ 5.61311547e-06,  4.87111596e-06,  6.55256693e-06, ...,
          3.87428576e-11,  2.51679355e

In [3]:
data = np.load('combined_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_merged_features.shape, Y.shape, groups_array.shape)

(73519, 20, 238) (73519,) (73519,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256)))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_merged_features, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_merged_features[train_index], X_merged_features[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.keras'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1


C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 21s 50ms/step - accuracy: 0.8452 - loss: 0.3456 - val_accuracy: 0.9344 - val_loss: 0.1672
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.9311 - loss: 0.1721 - val_accuracy: 0.9506 - val_loss: 0.1375
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9486 - loss: 0.1361 - val_accuracy: 0.9531 - val_loss: 0.1240
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9568 - loss: 0.1153 - val_accuracy: 0.9580 - val_loss: 0.1102
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9589 - loss: 0.1078 - val_accuracy: 0.9622 - val_loss: 0.1000
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9656 - loss: 0.0924 - val_accuracy: 0.9630 - val_loss: 0.0994
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.9676 - loss: 0.0870 - val_accuracy: 0.9661 - val_loss: 0.0912
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.9730 - loss: 0

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 23s 55ms/step - accuracy: 0.8475 - loss: 0.3481 - val_accuracy: 0.9256 - val_loss: 0.1886
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - accuracy: 0.9293 - loss: 0.1800 - val_accuracy: 0.9491 - val_loss: 0.1361
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.9458 - loss: 0.1399 - val_accuracy: 0.9573 - val_loss: 0.1116
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.9548 - loss: 0.1162 - val_accuracy: 0.9569 - val_loss: 0.1144
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - accuracy: 0.9597 - loss: 0.1031 - val_accuracy: 0.9611 - val_loss: 0.1024
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - accuracy: 0.9657 - loss: 0.0921 - val_accuracy: 0.9657 - val_loss: 0.0966
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.9679 - loss: 0.0841 - val_accuracy: 0.9645 - val_loss: 0.0910
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - accuracy: 0.9707 - loss: 0

...

Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 52s 151ms/step - accuracy: 0.8446 - loss: 0.3462 - val_accuracy: 0.9298 - val_loss: 0.1789
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 147ms/step - accuracy: 0.9278 - loss: 0.1776 - val_accuracy: 0.9486 - val_loss: 0.1368
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 148ms/step - accuracy: 0.9474 - loss: 0.1369 - val_accuracy: 0.9517 - val_loss: 0.1306
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 147ms/step - accuracy: 0.9543 - loss: 0.1182 - val_accuracy: 0.9591 - val_loss: 0.1112
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 147ms/step - accuracy: 0.9600 - loss: 0.1030 - val_accuracy: 0.9629 - val_loss: 0.1002
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 147ms/step - accuracy: 0.9680 - loss: 0.0857 - val_accuracy: 0.9644 - val_loss: 0.0982
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 147ms/step - accuracy: 0.9691 - loss: 0.0801 - val_accuracy: 0.9653 - val_loss: 0.0969
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 38s 148ms/step - accuracy: 0.9730 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 53s 136ms/step - accuracy: 0.8471 - loss: 0.3432 - val_accuracy: 0.9244 - val_loss: 0.1785
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 33s 126ms/step - accuracy: 0.9305 - loss: 0.1758 - val_accuracy: 0.9402 - val_loss: 0.1520
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 32s 124ms/step - accuracy: 0.9452 - loss: 0.1431 - val_accuracy: 0.9404 - val_loss: 0.1470
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 32s 123ms/step - accuracy: 0.9537 - loss: 0.1208 - val_accuracy: 0.9518 - val_loss: 0.1255
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 32s 124ms/step - accuracy: 0.9593 - loss: 0.1044 - val_accuracy: 0.9584 - val_loss: 0.1103
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 33s 126ms/step - accuracy: 0.9635 - loss: 0.0960 - val_accuracy: 0.9607 - val_loss: 0.0987
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 30s 117ms/step - accuracy: 0.9679 - loss: 0.0820 - val_accuracy: 0.9600 - val_loss: 0.1014
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 30s 117ms/step - accuracy: 0.9695 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 58s 162ms/step - accuracy: 0.8467 - loss: 0.3437 - val_accuracy: 0.9301 - val_loss: 0.1713
Epoch 2/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 156ms/step - accuracy: 0.9304 - loss: 0.1749 - val_accuracy: 0.9393 - val_loss: 0.1553
Epoch 3/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - accuracy: 0.9462 - loss: 0.1411 - val_accuracy: 0.9548 - val_loss: 0.1201
Epoch 4/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - accuracy: 0.9573 - loss: 0.1122 - val_accuracy: 0.9552 - val_loss: 0.1155
Epoch 5/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - accuracy: 0.9609 - loss: 0.1008 - val_accuracy: 0.9580 - val_loss: 0.1100
Epoch 6/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 153ms/step - accuracy: 0.9637 - loss: 0.0938 - val_accuracy: 0.9665 - val_loss: 0.0995
Epoch 7/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 154ms/step - accuracy: 0.9689 - loss: 0.0842 - val_accuracy: 0.9672 - val_loss: 0.0913
Epoch 8/200
259/259 ━━━━━━━━━━━━━━━━━━━━ 40s 154ms/step - accuracy: 0.9717 -

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9872,0.9870,0.9890,0.9880,0.9871,0.9851,0.9890,0.9743,0.9743
1,0.9863,0.9853,0.9890,0.9871,0.9861,0.9831,0.9890,0.9724,0.9724
2,0.9859,0.9847,0.9888,0.9868,0.9856,0.9825,0.9888,0.9716,0.9716
3,0.9864,0.9862,0.9883,0.9873,0.9863,0.9843,0.9883,0.9727,0.9727
4,0.9871,0.9878,0.9880,0.9879,0.9870,0.9860,0.9880,0.9740,0.9740
5,0.9869,0.9863,0.9893,0.9878,0.9868,0.9843,0.9893,0.9738,0.9738
6,0.9863,0.9875,0.9867,0.9871,0.9862,0.9857,0.9867,0.9724,0.9724
7,0.9850,0.9847,0.9872,0.9860,0.9849,0.9825,0.9872,0.9699,0.9699
8,0.9871,0.9863,0.9895,0.9879,0.9869,0.9843,0.9895,0.9740,0.9740
9,0.9863,0.9855,0.9888,0.9871,0.9861,0.9834,0.9888,0.9724,0.9724


In [13]:
metrics_df.round(4).to_csv('COMBINED_combined_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("COMBINED_combined_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9872,0.9870,0.9890,0.9880,0.9871,0.9851,0.9890,0.9743,0.9743
1,0.9863,0.9853,0.9890,0.9871,0.9861,0.9831,0.9890,0.9724,0.9724
2,0.9859,0.9847,0.9888,0.9868,0.9856,0.9825,0.9888,0.9716,0.9716
3,0.9864,0.9862,0.9883,0.9873,0.9863,0.9843,0.9883,0.9727,0.9727
4,0.9871,0.9878,0.9880,0.9879,0.9870,0.9860,0.9880,0.9740,0.9740
5,0.9869,0.9863,0.9893,0.9878,0.9868,0.9843,0.9893,0.9738,0.9738
6,0.9863,0.9875,0.9867,0.9871,0.9862,0.9857,0.9867,0.9724,0.9724
7,0.9850,0.9847,0.9872,0.9860,0.9849,0.9825,0.9872,0.9699,0.9699
8,0.9871,0.9863,0.9895,0.9879,0.9869,0.9843,0.9895,0.9740,0.9740
9,0.9863,0.9855,0.9888,0.9871,0.9861,0.9834,0.9888,0.9724,0.9724


In [16]:
(result.mean()*100).round(2)

Accuracy       98.64
Precision      98.61
Recall         98.85
F1             98.73
Sensitivity    98.63
Specificity    98.41
ROC_AUC        98.85
Kappa          97.28
MCC            97.28
dtype: float64

In [17]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])